In [1]:
file = r"C:\Users\roble\OneDrive\Documentos\GitHub\Recuperacion\ir25a-main\ir25a-main\data\12WEBCRAWLING\cheesy-zucchini-casserole-i"

In [2]:
from bs4 import BeautifulSoup

In [3]:
with open(file, 'r', encoding="utf-8") as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

In [4]:
title = soup.find("meta", {"property": "og:title"})["content"]
title

'Cheesy Zucchini Casserole'

In [6]:
ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
for ingredient in ingredients_section:
    print(ingredient.text.strip())

4 slices bread, cubed
¼ cup melted butter
2 cups cubed zucchini
1 large onion, finely chopped
1 large egg, beaten
1 teaspoon garlic salt or to taste
2 cups shredded Cheddar cheese


In [5]:
rating_div = soup.find("div", id="mm-recipes-review-bar__rating_1-0")
print('Rating: ',rating_div.text.strip())

Rating:  4.7


In [6]:
label = soup.find("div", class_="mm-recipes-details__label", string="Total Time:")
time_section = label.find_next_sibling("div", class_="mm-recipes-details__value")
print('Time: ', time_section.text.strip())

Time:  1 hr 15 mins


In [7]:
label = soup.find("div", class_="mm-recipes-details__label", string="Servings:")
serving_section = label.find_next_sibling("div", class_="mm-recipes-details__value")
print('Serving: ', serving_section.text.strip())

Serving:  4


In [8]:
ressume_class = soup.find("p", class_="article-subheading text-utility-300")
print('ressume: ',ressume_class.text.strip())

ressume:  This zucchini casserole with bread cubes, onion, and Cheddar is simple and easy to make for a deliciously cheesy vegetable side dish.


In [9]:
steps = soup.select("div#mm-recipes-steps__content_1-0 li")

print("Preparation Steps:")
for i, step in enumerate(steps, 1):
    paragraphs = step.find_all("p")
    step_text = " ".join(p.text.strip() for p in paragraphs if p.text.strip())
    print(f"Step {i}: {step_text}")

Preparation Steps:
Step 1: Gather all ingredients. Preheat the oven to 350 degrees F (175 degrees C). Dotdash Meredith Food Studios
Step 2: Place bread cubes in a medium bowl and pour melted butter over top. Add zucchini, onion, egg, and garlic salt; mix well. Dotdash Meredith Food Studios
Step 3: Transfer the mixture into a 9x13-inch baking dish and top with cheese. Dotdash Meredith Food Studios
Step 4: Cover and bake in the preheated oven for 30 minutes. Uncover the dish and bake for another 30 minutes.


In [10]:
nutrition_table = soup.find("table", class_="mm-recipes-nutrition-facts-summary__table")

print("Nutrition Facts Summary:")
if nutrition_table:
    rows = nutrition_table.find_all("tr")
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 2:
            value = cols[0].text.strip()
            label = cols[1].text.strip()
            print(f"{label}: {value}")
else:
    print("No se encontró la tabla nutricional.")

Nutrition Facts Summary:
Calories: 437
Fat: 32g
Carbs: 19g
Protein: 19g


In [11]:
recipe_links = soup. find_all("a", href=True)

recipe_urls = []
for link in recipe_links:
    href = link[ 'href']
    if "recipe" in href:
        recipe_urls.append(href)

print("Linked Recipes:")
for url in recipe_urls:
    print(url)

Linked Recipes:
https://www.allrecipes.com/authentication/login?regSource=3675&relativeRedirectUrl=%2Frecipe%2F20867%2Fcheesy-zucchini-casserole-i%2F
/account/add-recipe
https://www.myrecipes.com/favorites
https://www.allrecipes.com/authentication/logout?relativeRedirectUrl=%2Frecipe%2F20867%2Fcheesy-zucchini-casserole-i%2F
https://www.magazines.com/allrecipes-magazine.html?utm_source=allrecipes.com&utm_medium=owned&utm_campaign=i111arr1w2661
https://www.magazines.com/allrecipes-magazine.html
https://www.allrecipes.com/recipes/17562/dinner/
https://www.allrecipes.com/recipes/17057/everyday-cooking/more-meal-ideas/5-ingredients/main-dishes/
https://www.allrecipes.com/recipes/15436/everyday-cooking/one-pot-meals/
https://www.allrecipes.com/recipes/1947/everyday-cooking/quick-and-easy/
https://www.allrecipes.com/recipes/455/everyday-cooking/more-meal-ideas/30-minute-meals/
https://www.allrecipes.com/recipes/17889/everyday-cooking/family-friendly/family-dinners/
https://www.allrecipes.com/

In [12]:
import json
import requests
import time
import re

In [22]:
# Simula un navegador web para evitar bloqueos por scraping
headers = {"User-Agent": "Mozilla/5.0"}
url = "https://www.allrecipes.com/recipe/20867/cheesy-zucchini-casserole-i/"
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.content, "html.parser")

In [26]:
from urllib.parse import urljoin

headers = {"User-Agent": "Mozilla/5.0"}
base_url = "https://www.allrecipes.com/search"
query = "egg"
max_recipes = 99
step = 24 
recipe_urls = set()
offset = 0

In [28]:
while len(recipe_urls) < max_recipes:
    url = f"{base_url}?q={query}&offset={offset}"
    print(f"\n🔎 Buscando recetas en: {url}")
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = soup.find_all("a", href=True)
    
    found = 0
    for link in links:
        href = link['href']
        full_url = urljoin(url, href)

        if (
            "/recipe/" in full_url and 
            full_url.startswith("https://www.allrecipes.com/recipe/") and 
            full_url not in recipe_urls
        ):
            recipe_urls.add(full_url)
            found += 1

        if len(recipe_urls) >= max_recipes:
            break

    if found == 0:
        print("⚠️ No se encontraron más recetas. Finalizando búsqueda.")
        break

    offset += step

print("\n✅ Recetas recopiladas:")
for i, recipe in enumerate(sorted(recipe_urls), 1):
    print(f"{i:02d}. {recipe}")

print(f"\n📦 Total recopilado: {len(recipe_urls)} recetas.")



✅ Recetas recopiladas:
01. https://www.allrecipes.com/recipe/103767/jens-heavenly-egg-salad/
02. https://www.allrecipes.com/recipe/104804/curried-egg-sandwiches/
03. https://www.allrecipes.com/recipe/105904/cheeseburger-egg-rolls/
04. https://www.allrecipes.com/recipe/11288/egg-free-chocolate-chip-pumpkin-cookies/
05. https://www.allrecipes.com/recipe/11586/egg-free-low-fat-oatmeal-cookies/
06. https://www.allrecipes.com/recipe/115965/egg-drop-soup-better-than-restaurant-quality/
07. https://www.allrecipes.com/recipe/11991/egg-noodles/
08. https://www.allrecipes.com/recipe/12374/bacon-and-egg-breakfast-tarts/
09. https://www.allrecipes.com/recipe/12543/egg-custard-pie-iii/
10. https://www.allrecipes.com/recipe/12696/grandmas-egg-custard-pie/
11. https://www.allrecipes.com/recipe/128589/oyakodon-japanese-chicken-and-egg-rice-bowl/
12. https://www.allrecipes.com/recipe/12876/egg-rolls/
13. https://www.allrecipes.com/recipe/132372/bacon-egg-and-cheese-strata/
14. https://www.allrecipes.c

In [30]:
import os
import time

save_dir = r"\data\12WEBCRAWLING\recetas"
os.makedirs(save_dir, exist_ok=True)

In [31]:
for i, url in enumerate(sorted(recipe_urls), 1):
    print(f"\n💾 Guardando receta {i:02d} de {len(recipe_urls)}")
    print(f"🔗 URL: {url}")
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        filename = os.path.join(save_dir, f"receta_{i:02d}.html")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(response.text)
        print(f"✅ Guardado en: {filename}")
        time.sleep(1.5)  # Evitar bloqueo
    except Exception as e:
        print(f"❌ Error al guardar receta {i:02d}: {e}")



💾 Guardando receta 01 de 99
🔗 URL: https://www.allrecipes.com/recipe/103767/jens-heavenly-egg-salad/
✅ Guardado en: \data\12WEBCRAWLING\recetas\receta_01.html

💾 Guardando receta 02 de 99
🔗 URL: https://www.allrecipes.com/recipe/104804/curried-egg-sandwiches/
✅ Guardado en: \data\12WEBCRAWLING\recetas\receta_02.html

💾 Guardando receta 03 de 99
🔗 URL: https://www.allrecipes.com/recipe/105904/cheeseburger-egg-rolls/
✅ Guardado en: \data\12WEBCRAWLING\recetas\receta_03.html

💾 Guardando receta 04 de 99
🔗 URL: https://www.allrecipes.com/recipe/11288/egg-free-chocolate-chip-pumpkin-cookies/
✅ Guardado en: \data\12WEBCRAWLING\recetas\receta_04.html

💾 Guardando receta 05 de 99
🔗 URL: https://www.allrecipes.com/recipe/11586/egg-free-low-fat-oatmeal-cookies/
✅ Guardado en: \data\12WEBCRAWLING\recetas\receta_05.html

💾 Guardando receta 06 de 99
🔗 URL: https://www.allrecipes.com/recipe/115965/egg-drop-soup-better-than-restaurant-quality/
✅ Guardado en: \data\12WEBCRAWLING\recetas\receta_06.htm

In [32]:
import pandas as pd

# Directorio donde están los archivos
html_dir = r"\data\12WEBCRAWLING\recetas"
recetas_data = []

In [33]:
for i in range(1, 100):  # ajusta el rango según los archivos guardados
    print(f"\n📂 Procesando receta {i:02d}...")
    try:
        file_path = os.path.join(html_dir, f"receta_{i:02d}.html")
        with open(file_path, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file.read(), "html.parser")
        
        # Título
        title_tag = soup.find("meta", {'property': 'og:title'})
        title = title_tag["content"].strip() if title_tag else "N/A"

        # Rating
        rating_tag = soup.find("div", class_="mm-recipes-review-bar__rating")
        rating = rating_tag.text.strip() if rating_tag else "N/A"

        # Ingredientes
        ingredient_tags = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
        ingredients_list = [ing.text.strip() for ing in ingredient_tags]

        # Porciones y tiempo total
        servings, total_time = "", ""
        for s in soup.find_all("div", class_="mm-recipes-details__item"):
            if "Servings" in s.text:
                v = s.find("div", class_="mm-recipes-details__value")
                servings = v.text.strip() if v else ""
            if "Total Time" in s.text:
                v = s.find("div", class_="mm-recipes-details__value")
                total_time = v.text.strip() if v else ""

        # Descripción
        desc_tags = soup.find_all("p", class_="article-subheading text-utility-300")
        description = " ".join([d.text.strip() for d in desc_tags])

        # Pasos
        steps_tags = soup.find_all("li", class_="mntl-sc-block-group--LI")
        steps_list = []
        for step in steps_tags:
            p = step.find("p")
            if p:
                steps_list.append(p.text.strip())

        # Nutrición
        nutrition_rows = soup.find_all("tr", class_="mm-recipes-nutrition-facts-summary__table-row")
        nutrition = {}
        for row in nutrition_rows:
            label = row.find("td", class_="text-body-100")
            value = row.find("td", class_="text-body-100-prominent")
            if label and value:
                nutrition[label.text.strip()] = value.text.strip()

        # Guardar resultados
        recetas_data.append({
            "Título": title,
            "Rating": rating,
            "Porciones": servings,
            "Tiempo total": total_time,
            "Ingredientes": ingredients_list,
            "Descripción": description,
            "Pasos": steps_list,
            "Nutrición": nutrition
        })

        print(f"✅ Receta {i:02d} procesada correctamente.")

    except Exception as e:
        print(f"❌ Error en receta {i:02d}: {e}")



📂 Procesando receta 01...
✅ Receta 01 procesada correctamente.

📂 Procesando receta 02...
✅ Receta 02 procesada correctamente.

📂 Procesando receta 03...
✅ Receta 03 procesada correctamente.

📂 Procesando receta 04...
✅ Receta 04 procesada correctamente.

📂 Procesando receta 05...
✅ Receta 05 procesada correctamente.

📂 Procesando receta 06...
✅ Receta 06 procesada correctamente.

📂 Procesando receta 07...
✅ Receta 07 procesada correctamente.

📂 Procesando receta 08...
✅ Receta 08 procesada correctamente.

📂 Procesando receta 09...
✅ Receta 09 procesada correctamente.

📂 Procesando receta 10...
✅ Receta 10 procesada correctamente.

📂 Procesando receta 11...
✅ Receta 11 procesada correctamente.

📂 Procesando receta 12...
✅ Receta 12 procesada correctamente.

📂 Procesando receta 13...
✅ Receta 13 procesada correctamente.

📂 Procesando receta 14...
✅ Receta 14 procesada correctamente.

📂 Procesando receta 15...
✅ Receta 15 procesada correctamente.

📂 Procesando receta 16...
✅ Receta 16 p

In [34]:
df = pd.DataFrame(recetas_data)
csv_path = "recetas_extraidas.csv"
df.to_csv(csv_path, index=False)
print(f"\n✅ CSV creado con éxito: {csv_path}")
print(f"🔢 Total de recetas guardadas: {len(df)}")



✅ CSV creado con éxito: recetas_extraidas.csv
🔢 Total de recetas guardadas: 99


In [35]:
print("\n📋 Mostrando el DataFrame completo de recetas extraídas:\n")
display(df)  # Si usas Jupyter, esto hace que se vea bonito; si no, simplemente usa df



📋 Mostrando el DataFrame completo de recetas extraídas:



,Título,Rating,Porciones,Tiempo total,Ingredientes,Descripción,Pasos,Nutrición
0,Jen's Heavenly Egg Salad,4.5,4,20 mins,"[6 eggs, ¼ cup mayonnaise, ¼ cup chopped green...","The tang from Dijon mustard, freshness from le...",[Place eggs in a saucepan and cover with water...,"{'Calories': '212', 'Fat': '18g', 'Carbs': '3g..."
1,Curried Egg Sandwiches,4.3,4,15 mins,"[4 hard-cooked eggs, peeled and chopped, ½ cu...",Delicious curried egg salad sandwiches guarant...,"[Gather your ingredients., Mix together mayonn...","{'Calories': '410', 'Fat': '29g', 'Carbs': '27..."
2,Cheeseburger Egg Rolls,4.7,15,50 mins,"[1 pound ground beef, 1 onion, chopped, 1 cup...","I had something similar at a restaurant, threw...",[Place the ground beef and onion in a skillet ...,"{'Calories': '189', 'Fat': '8g', 'Carbs': '19g..."
3,Egg Free Chocolate Chip Pumpkin Cookies,4.6,48,,"[2 cups white sugar, 1 cup shortening, 1 (15 o...",This is a wonderful recipe when there are no e...,[Preheat oven to 375 degrees F (190 degrees C)...,"{'Calories': '146', 'Fat': '7g', 'Carbs': '22g..."
4,Egg-Free Low-Fat Oatmeal Cookies,3.8,24,25 mins,"[¾ cup applesauce, ¼ cup packed brown sugar, ¼...",These low-fat oatmeal cookies made with apples...,[Preheat the oven to 350 degrees F ( 175 degre...,"{'Calories': '77', 'Fat': '2g', 'Carbs': '14g'..."
...,...,...,...,...,...,...,...,...
94,Awesome Egg Rolls,4.5,10,1 hr,"[1 tsp vegetable oil, or as needed, 1 large eg...",This shrimp egg roll recipe takes some time to...,"[Gather all ingredients., Mix cabbage, bean sp...","{'Calories': '1268', 'Fat': '110g', 'Carbs': '..."
95,One Egg Lemon Pound Cake,4.0,10,,"[2 cups all-purpose flour, 2 teaspoons baking ...",My Grandmother used to make this and it quickl...,[Preheat oven to 350 degrees F (175 degrees C)...,"{'Calories': '272', 'Fat': '10g', 'Carbs': '41..."
96,Awesome Egg Salad with a Kick,4.5,8,15 mins,"[12 large hard-cooked eggs, peeled and chopped...","Try this egg salad with sour cream, crumbled b...","[Mix eggs, mayonnaise, sour cream, bacon bits,...","{'Calories': '174', 'Fat': '11g', 'Carbs': '7g..."
97,Egg Salad Sandwiches,4.2,4,8 hrs 10 mins,"[8 hard-cooked eggs, diced, 1 cup mayonnaise,...","Mustard powder, dill weed, garlic powder, and ...","[In a bowl, gently mix the eggs, mayonnaise, o...","{'Calories': '701', 'Fat': '56g', 'Carbs': '32..."


In [37]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

df = pd.read_csv('recetas_extraidas.csv')

df['contexto'] = (
    "Título: "      + df['Título'].astype(str) + "\n"
    "Rating: "      + df['Rating'].astype(str) + "\n"
    "Porciones: "   + df['Porciones'].astype(str) + "\n"
    "Tiempo total: "+ df['Tiempo total'].astype(str) + "\n"
    "Ingredientes: "+ df['Ingredientes'].astype(str) + "\n"
    "Descripción: " + df['Descripción'].astype(str) + "\n"
    "Pasos: "       + df['Pasos'].astype(str) + "\n"
    "Nutrición: "   + df['Nutrición'].astype(str)
)

print("✅ Columna 'contexto' creada. Ejemplo de contexto:\n")
print(df['contexto'].iloc[0][:700])  # Imprime un ejemplo truncado



✅ Columna 'contexto' creada. Ejemplo de contexto:

Título: Jen's Heavenly Egg Salad
Rating: 4.5
Porciones: 4
Tiempo total: 20 mins
Ingredientes: ['6 eggs', '¼ cup mayonnaise', '¼ cup chopped green onions', '½ lemon, juiced', '1 teaspoon Dijon mustard', '½ teaspoon prepared yellow mustard', 'salt and ground black pepper to taste']
Descripción: The tang from Dijon mustard, freshness from lemon, and liveliness from watercress make Jen's heavenly egg salad a must! This recipe tastes best when paired with wheat bread, watercress, and avocado, but I can eat it alone.
Pasos: ['Place eggs in a saucepan and cover with water. Bring to a boil, remove from heat, cover, and let eggs stand in hot water for 10 to 12 minutes. Remove eggs from hot water, coo


In [39]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['contexto'].tolist())
embeddings = np.array(embeddings).astype('float32')
print(f"✅ Embeddings generados: {embeddings.shape}")

✅ Embeddings generados: (99, 384)


In [40]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print(f"✅ Índice FAISS creado: {type(index).__name__}")
print(f"🔢 Total de vectores indexados: {index.ntotal}")
print(f"📐 Dimensión del embedding: {embeddings.shape[1]}")

✅ Índice FAISS creado: IndexFlatL2
🔢 Total de vectores indexados: 99
📐 Dimensión del embedding: 384


In [41]:
query = "egg salad"  # Cambia aquí tu consulta
query_emb = model.encode([query]).astype('float32')
k = 5

distances, indices = index.search(query_emb, k)

print(f"\n🔎 Resultados para la consulta: \"{query}\"\n")
for rank, (idx, dist) in enumerate(zip(indices[0], distances[0]), 1):
    print(f"{rank}. Título: {df.loc[idx, 'Título']}")
    print(f"   Distancia: {dist:.4f}")
    print(f"   URL: {df.loc[idx, 'URL']}" if 'URL' in df.columns else "")
    print("---")



🔎 Resultados para la consulta: "egg salad"

1. Título: Polish Egg Salad
   Distancia: 0.5681

---
2. Título: Egg Salad
   Distancia: 0.5922

---
3. Título: Egg Salad Sandwiches
   Distancia: 0.6560

---
4. Título: Egg Salad with Olives
   Distancia: 0.6990

---
5. Título: Jen's Heavenly Egg Salad
   Distancia: 0.6996

---


In [42]:
similarity_df = pd.DataFrame({
    'index': indices[0],
    'distance': distances[0]
})

# Puedes agregar más columnas de interés si quieres
similarity_df['Título'] = df.iloc[similarity_df['index']]['Título'].values
similarity_df['contexto'] = df.iloc[similarity_df['index']]['contexto'].values
if 'URL' in df.columns:
    similarity_df['URL'] = df.iloc[similarity_df['index']]['URL'].values

similarity_df = similarity_df.sort_values(by='distance').reset_index(drop=True)
display(similarity_df[['Título', 'distance', 'URL', 'contexto']] if 'URL' in similarity_df.columns else similarity_df)

,index,distance,Título,contexto
0,72,0.568148,Polish Egg Salad,Título: Polish Egg Salad\nRating: 4.4\nPorcion...
1,23,0.592215,Egg Salad,Título: Egg Salad\nRating: 4.0\nPorciones: 4\n...
2,97,0.655958,Egg Salad Sandwiches,Título: Egg Salad Sandwiches\nRating: 4.2\nPor...
3,22,0.699042,Egg Salad with Olives,Título: Egg Salad with Olives\nRating: 4.3\nPo...
4,0,0.699581,Jen's Heavenly Egg Salad,Título: Jen's Heavenly Egg Salad\nRating: 4.5\...


In [43]:
contexto_similar = "\n\n".join(similarity_df['contexto'].tolist())
print(contexto_similar)

Título: Polish Egg Salad
Rating: 4.4
Porciones: 6
Tiempo total: 10 mins
Ingredientes: ['1 (8 ounce) package cream cheese, softened', '1 tablespoon butter, softened, or more to taste', '4  hard-boiled eggs, chopped', '1 small onion, chopped', 'salt and ground black pepper to taste', '1 tablespoon chopped fresh parsley, or to taste']
Descripción: This creamy-style egg salad is quite common in Poland. It has a similar consistency as egg salad with mayo but it is made with cream cheese. Lovely in a sandwich with fresh cress.
Pasos: ['Combine cream cheese and butter in a bowl and mash with a fork. If mixture is too thick, add more softened butter. Mix in eggs and onion. Season with salt and pepper; serve with parsley.']
Nutrición: {'Calories': '204', 'Fat': '19g', 'Carbs': '3g', 'Protein': '7g'}

Título: Egg Salad
Rating: 4.0
Porciones: 4
Tiempo total: 30 mins
Ingredientes: ['8 large eggs', '½  red onion, minced', '2 tablespoons prepared Dijon-style mustard', '1 tablespoon mayonnaise', '1 t

In [ ]:
from openai import OpenAI
API_KEY = ""
client = OpenAI(api_key=API_KEY)

completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "Eres una aplicación de Retrieval Augmented Generation (RAG) "
                "que responde SIEMPRE en español. Solo puedes responder usando la información"
                "contenida en el contexto proporcionado a continuación. Si la respuesta no "
                "está en el contexto, responde claramente: "
                "No sé la respuesta porque no está en el contexto"
            },
            {
                "role": "user",
                "content": f"""El usuario esta preguntanddo algo sobre recetas de cocina. Tu tarea es " 
                responde a la pregunta: "{query}"

DOCUMENTOS DE REFERENCIA:
{contexto_similar}

INSTRUCCIONES:
- Responde solo con información que esté explícitamente en los documentos
- Si no hay información suficiente, di "No encuentro información suficiente en los documentos proporcionados"""
            }
        ],
        max_tokens=2000,
        temperature=0.1
    )
print("\n🤖 Respuesta del modelo:"
      f"\n{completion.choices[0].message.content.strip()}\n")



🤖 Respuesta del modelo:
La ensalada de huevo (Egg Salad) es una receta que generalmente incluye huevos duros, cebolla roja, mostaza, mayonesa, eneldo seco, pimentón, sal y pimienta. Aquí tienes una receta básica:

**Ingredientes:**
- 8 huevos grandes
- ½ cebolla roja, picada
- 2 cucharadas de mostaza tipo Dijon
- 1 cucharada de mayonesa
- 1 cucharadita de eneldo seco
- 1 cucharadita de pimentón
- Sal y pimienta al gusto

**Pasos:**
1. Coloca los huevos en una cacerola y cúbrelos con agua. Lleva a ebullición, retira del fuego y deja los huevos en el agua caliente durante 15 minutos.
2. Retira los huevos del agua caliente, enfría bajo agua fría, pélalos y pícalos.
3. Mezcla los huevos picados, la cebolla, la mostaza, la mayonesa, el eneldo, el pimentón, la sal y la pimienta en un tazón grande. Machaca bien con un tenedor o cuchara de madera.
4. Sirve en pan como un sándwich o sobre lechuga crujiente como ensalada.

Esta receta tiene un tiempo total de 30 minutos y rinde para 4 porciones